In [ ]:
from pyspark.sql.functions import udf, col, window
from pyspark.sql.types import StringType
from util import kafkaConsumer
import pandas as pd
from operations import SparkInst


In [5]:
spark_job=SparkInst("AWAS SYSTEM", 5, "violations")

In [8]:
import pandas as pd

df_pd = pd.read_csv("data/camera.csv")
if '_id' in df_pd.columns:
    df_pd.drop(columns=['_id'], inplace=True)
spark_df = spark_job.get_session().createDataFrame(df_pd)

# Step 3: Broadcast your speed limit map
speed_limit_map = {row['camera_id']: row['speed_limit'] for row in spark_df.select("camera_id", "speed_limit").collect()}
broadcast_map = spark_job.essentialData_broadcast(spark_df)
# Step 4: Define your UDF using broadcast variable
def mark_speeding(camera_id:str, speed:float, ops:str):
    limit = broadcast_map.value.get(camera_id)
    if limit is not None and ops == "instant":
        return "INSTANT_VIOLATION" if speed > limit else None
    elif limit is not None and ops == "average":
        return "AVERAGE_VIOLATION" if speed > limit else None
    return "NONE"

speeding_udf = udf(mark_speeding, StringType())

# Step 5: Apply UDF to each streaming dataframe
def add_speed_flag(df, ops:str ):
    return df.withColumn("speed_flag", speeding_udf(col("camera_id"), col("speed_reading"), ops))

/opt/conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/opt/conda/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [ ]:
from pyspark.sql.functions import expr, col

# Attach Kafka streams
stream_a = spark_job.attach_kafka_stream("topic_events_a", "9092", "5 minutes")
stream_b = spark_job.attach_kafka_stream("topic_events_b", "9092", "5 minutes")
stream_c = spark_job.attach_kafka_stream("topic_events_c", "9092", "5 minutes")

# Apply transformation
stream_a_flagged = add_speed_flag(stream_a, "instant")
stream_b_flagged = add_speed_flag(stream_b, "instant")
stream_c_flagged = add_speed_flag(stream_c, "instant")

# Duplicate stream_b
dup1_b = stream_b_flagged
dup2_b = stream_b_flagged

# Rename timestamps to avoid conflict
stream_a_renamed = stream_a_flagged.withColumnRenamed("timestamp", "timestamp_start")
stream_b1_renamed = dup1_b.withColumnRenamed("timestamp", "timestamp_end")
stream_c_renamed = stream_c_flagged.withColumnRenamed("timestamp", "timestamp_start")
stream_b2_renamed = dup2_b.withColumnRenamed("timestamp", "timestamp_end")



# Apply watermarks
stream_a_watermarked = stream_a_renamed.withWatermark("timestamp_a", "5 minutes")
stream_b1_watermarked = stream_b1_renamed.withWatermark("timestamp_b", "5 minutes")
stream_c_watermarked = stream_c_renamed.withWatermark("timestamp_c", "5 minutes")
stream_b2_watermarked = stream_b2_renamed.withWatermark("timestamp_b2", "5 minutes")

# Join A and B on car_plate and timestamp condition
ab_join = stream_b1_watermarked.join(
    stream_a_watermarked,
    expr("""
        stream_b1_watermarked.car_plate = stream_a_watermarked.car_plate AND
        timestamp_a > timestamp_b AND
        timestamp_a <= timestamp_b + interval 10 minutes
    """),
    "inner"
).withColumn(
    "avg_measure_speed",
    (col("measure_speed_a") + col("measure_speed_b")) / 2
).withColumn("speed_flag", speeding_udf(col("camera_id"), col("speed_reading"), "average"))

# Join B and C on car_plate and timestamp condition
bc_join = stream_b2_watermarked.join(
    stream_c_watermarked,
    expr("""
        stream_b2_watermarked.car_plate = stream_c_watermarked.car_plate AND
        timestamp_c > timestamp_b2 AND
        timestamp_c <= timestamp_b2 + interval 10 minutes
    """),
    "inner"
).withColumn(
    "avg_measure_speed",
    (col("measure_speed_a") + col("measure_speed_b")) / 2
).withColumn("speed_flag", speeding_udf(col("camera_id"), col("speed_reading"), "average"))


In [ ]:
ab_join.union(bc_join, window(col("timestamp"), "24 hours")).dropna().select()
